# SECTION 1: Introduction

Welcome to my NLP project using GPT-3 and Seinfeld. The goal is to allow the user to prompt GPT3 Seinfeld bot with a situation and have it respond with an AI-generated plot synopsis.

Example:
> Prompt: "Trying to connect to WiFi"  
> Response: "When the WiFi George usually steals suddenly has a password, he becomes addicted to trying to "hack" in. J: 'Just get your own!' G: 'NEVER' "

This example response is from the @ModernSeinfeld twitter feed.




